# Chapter 18: Variational Quantum Linear Solver

Variational Quantum Linear Solver (VQLS) and parametric circuits.

---

**Prerequisites:**
- See `Chapter02_QuantumSoftware.ipynb` for installation instructions


In [12]:
# Setup and imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from qiskit_aer import Aer
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import real_amplitudes as RealAmplitudes
from qiskit.quantum_info import SparsePauliOp, Statevector,Operator
from qiskit.primitives import StatevectorEstimator
from scipy.optimize import minimize
print('Setup complete!')

Setup complete!


## VQLS via Norm Minimization

In [ ]:
# Predefined Pauli expansion
# 1. Define the Problem
# A = 0.5Z + 0.5X, b = |0>
matrix_a = SparsePauliOp.from_list([("Z", 0.5), ("X", 0.5)])
num_qubits = 1

# 2. Setup the Ansatz
ansatz = RealAmplitudes(num_qubits, reps=2)
estimator = StatevectorEstimator()

# 3. Define the Cost Function (Global Norm)
def cost_func(params):
    bound_ansatz = ansatz.assign_parameters(params)
    state = Statevector(bound_ansatz)
    
    # Compute |A|u> - |b>
    A_u = state.evolve(matrix_a).data
    b_vec = np.zeros(2**num_qubits)
    b_vec[0] = 1.0
    
    residual = A_u - b_vec
    cost = np.real(np.vdot(residual, residual))
    return float(cost)

# 4. Optimize
initial_theta = np.random.rand(ansatz.num_parameters)
res = minimize(cost_func, initial_theta, method='COBYLA')

print(f"Optimal Parameters: {res.x}")
print(f"Final Cost: {res.fun}")

# --- 1. Get the Quantum Solution ---
final_circuit = ansatz.assign_parameters(res.x)
quantum_solution = Statevector(final_circuit).data

# --- 2. Get the Exact Classical Solution ---
# Convert our SparsePauliOp A to a dense matrix
A_matrix = Operator(matrix_a).data
# Define b as a vector (assuming |b> = |0>, so [1, 0, 0...])
b_vector = np.zeros(2**num_qubits)
b_vector[0] = 1 

# Solve Ax = b classically
exact_solution = np.linalg.solve(A_matrix, b_vector)
# Normalize the exact solution to unit length for comparison
exact_solution_norm = exact_solution / np.linalg.norm(exact_solution)

# Calculate Fidelity (how close the two vectors are)
print("Exact Solution (Normalized):", exact_solution_norm)
print("Quantum Solution:", quantum_solution)
fidelity = np.abs(np.vdot(exact_solution_norm, quantum_solution))**2
print(f"Solution Fidelity: {fidelity:.4f}")

Optimal Parameters: [0.78841508 0.69092297 0.09166113]
Final Cost: 0.08578644490164561
Exact Solution (Normalized): [0.70710678+0.j 0.70710678-0.j]
Quantum Solution: [0.70703506+0.j 0.7071785 +0.j]
Solution Fidelity: 1.0000


## Ax = b Examples

In [ ]:
example = 1
if (example == 1):
	A = np.array([[1,0],[0,0.75]]) 
	b = np.array([0,1])
elif (example == 2):
	A = np.array([[2,-1],[-1,2]])
	b = np.array([1,1])/np.sqrt(2)
elif (example == 3):
	A = np.array([[1,0,0,-0.5],[0,1,0,0],[0,0,1,0],[-0.5,0,0,1]])
	b = np.array([1,0,0,0])
elif (example == 4):
	A = np.array([[1.5,0.5],[0.5,1.5]])
	b = np.array([1,0])
elif (example == 5):
	p = 2
	A = np.array([[5*(10**p),-1],[-1,5]])
	b = np.array([1,0])
elif (example == 6):
	N = 4
	values = [-np.ones(N-1),2*np.ones(N),-np.ones(N-1)]
	A = diags(values,[-1,0,1]).toarray()
	b = np.zeros(N)
	b[0] = 1

print("A:\n", A)
print("b:\n", b)

A:
 [[1.   0.  ]
 [0.   0.75]]
b:
 [0 1]


## VQLS using Pauli Decompostion and Norm Minimization